In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('simpsons.tsv', sep='\t')
CHARACTER_NAME = 'Homer Simpson'
print(len(data))
data.head()
data[data['line'] == 'Man, buffalo are easy to kill.']
data.iloc[60430:60441]

140751


,name,line
60430,Marge Simpson,NaN
60431,Paul Bunyan,We've been together a long time now. When are ...
60432,Marge Simpson,Soon. I just need a few more yoga classes.
60433,Marge Simpson,"Oh, look! A shooting star!"
60434,Professor Jonathan Frink,"That meteor's headed straight for us, with the..."
60435,Rev. Timothy Lovejoy,God has sent this fiery killrock to show us hi...
60436,Villagers,"No, we're gonna die! / Oh no! /"
60437,Mayor Joe Quimby,There's only one man who can save us...
60438,Homer Simpson,"Man, buffalo are easy to kill."
60439,Paul Bunyan,"Oh, I get it. When I'm crushing and killing yo..."


In [3]:
contexted = []

n = 1

for i in data[data.name == CHARACTER_NAME].index:
  if i < n:
    continue
  row = []
  prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 7 previous responces  
  for j in range(i, prev, -1):
    row.append(data.line[j])
  contexted.append(row)

# print(len(contexted))

columns = ['completion', 'prompt'] 

df = pd.DataFrame.from_records(contexted, columns=columns)

In [4]:
len(df)

29842

In [5]:
df = df.dropna()
len(df)

26423

In [6]:
df.sample(6)

,completion,prompt
25780,"And if it's such a ""Good Book,"" how come there...","Padre, can I be honest with you? I've sucked e..."
14342,You might hear from your friends or co-workers...,"Kids, we want to talk to you about something."
4786,I'll never be able to do this trick... never.,Okay.
12618,And his fondness for the guitar.,Wow. The artist really captured Jimi's passion...
17983,"Shut up, Flanders. But some things I could nev...","Well, that is true. In fact--"
7542,Me? But you were doing such a great job.,"Uh-huh. Half a brain, huh? Well, you know what..."


In [7]:
df.to_csv('simpsons_openai.tsv', sep='\t',index=False)

## Calculate the number of tokens

In [21]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("davinci")

In [61]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")

    num_tokens = 0
    for message in messages:
        for value in message:
            num_tokens += len(encoding.encode(value))
    return num_tokens


In [62]:
messages = pd.read_json('simpsons_openai_prepared.jsonl', lines=True)
messages = messages.values.tolist()

In [63]:
model = 'davinci'
cost_per_token = 0.03 / 1000.0
num_tokens = num_tokens_from_messages(messages, model)
print(f"{num_tokens} prompt tokens counted.")
cost = num_tokens * cost_per_token
print(f"The estimated cost is {cost}")

869921 prompt tokens counted.
The estimated cost is 26.09763


In [64]:
model = 'babbage'
cost_per_token = 0.0006 / 1000.0
num_tokens = num_tokens_from_messages(messages, model)
print(f"{num_tokens} prompt tokens counted.")
cost = num_tokens * cost_per_token
print(f"The estimated cost is {cost}")

869921 prompt tokens counted.
The estimated cost is 0.5219526


In [65]:
model = 'curie'
cost_per_token = 0.003 / 1000.0
num_tokens = num_tokens_from_messages(messages, model)
print(f"{num_tokens} prompt tokens counted.")
cost = num_tokens * cost_per_token
print(f"The estimated cost is {cost}")

869921 prompt tokens counted.
The estimated cost is 2.609763


In [69]:
message_example = messages[0][0]
print(message_example)
enc = encoding.encode(message_example)
len(enc)

Goodbye, Lisa honey. It'll be okay. Just read the note.

###




20